In [57]:
import numpy as np
import pandas as pd

*Importando o arquivo*

In [58]:
# Caminho para o arquivo CSV
caminho = r'C:\Users\igor cardoso\Desktop\Faculdade\df_t.csv'

# Leitura do arquivo CSV
df = pd.read_csv(caminho)

# Exibe a tabela
df.head()


,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success
0,bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,2018-12-26,15:33:35,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,50e9a8665b62c8d68bccc77c7c92431a1aa26ccbd38ed4...,0,0,8527a891e224136950ff32ca212b45bc93f69fbb801c3b...,1,89.09,1
1,5432f12612dd5d749b3be880e779989cf63b5efa4bcc4e...,37228485e0dc83d84d1bcd1bef3dc632301bf6cb22c8b5...,2018-12-05,15:07:57,10e4e7caf8b078429bb1c80b1a10118ac6f963eff098fd...,e6d41d208672a4e50b86d959f4a6254975e6fb9b088116...,0,0,36ebe205bcdfc499a25e6923f4450fa8d48196ceb4fa0c...,1,155.97,1
2,fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,2018-12-21,18:41:54,7688b6ef52555962d008fff894223582c484517cea7da4...,8c1f1046219ddd216a023f792356ddf127fce372a72ec9...,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,121.99,1
3,4dc44a6dd592b702feccb493d192210c86965aee684529...,ab3251a2be0f69713b8f97b0e9d1579e31551f4fd4facf...,2018-12-06,14:01:38,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,d6acb3c1a79e57bcc03d976cb4d98f56edccd4cf426392...,0,0,5f9c4ab08cac7457e9111a30e4664920607ea2c115a143...,1,55.22,1
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,ceea0de820a6379f2c4215bddaec66c33994b304607e56...,2021-02-23,20:08:25,7688b6ef52555962d008fff894223582c484517cea7da4...,23765fc69c4e3c0b10f5d15471dc2245e2a19af16b513f...,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1


In [59]:

df_curado = df

# 1. Garantir que os IDs são strings
df_curado['nk_ota_localizer_id'] = df_curado['nk_ota_localizer_id'].astype(str)
df_curado['fk_contact'] = df_curado['fk_contact'].astype(str)

# 2. Criar o campo datetime unificando data e hora
df_curado['data_hora_compra'] = pd.to_datetime(
    df_curado['date_purchase'] + ' ' + df_curado['time_purchase'], errors='coerce'
)

# 3. Trocar "0" por np.nan nos campos de retorno (considerando que são strings)
campos_com_zero_para_nulo = [
    'place_origin_return', 
    'place_destination_return'
]

for col in campos_com_zero_para_nulo:
    df_curado[col] = df_curado[col].replace("0", np.nan)

# Trocar "0" e "1" por np.nan na fk_return_ota_bus_company
df_curado['fk_return_ota_bus_company'] = df_curado['fk_return_ota_bus_company'].replace(["0", "1"], np.nan)

# 4. Criar coluna "classificacao_viagem"
df_curado['classificacao_viagem'] = df_curado['place_origin_return'].apply(
    lambda x: 'ida_e_volta' if pd.notna(x) else 'ida'
)

# 5. Garantir que os tipos de gmv e tickets estão corretos
df_curado['gmv_success'] = df_curado['gmv_success'].astype(float)
df_curado['total_tickets_quantity_success'] = df_curado['total_tickets_quantity_success'].astype(int)

# 6. Remover colunas antigas de data e hora
df_curado.drop(['date_purchase', 'time_purchase'], axis=1, inplace=True)

In [60]:
# 7. Criar colunas adicionais

# tipo_compra: individual ou coletiva
df_curado['tipo_compra'] = df_curado['total_tickets_quantity_success'].apply(
    lambda x: 'individual' if x == 1 else 'coletiva'
)

# sem_retorno_flag: True se retorno for nulo
df_curado['sem_retorno_flag'] = df_curado['place_origin_return'].isnull()

# compra_dia_util: True se compra em dia de semana (segunda a sexta)
df_curado['compra_dia_util'] = df_curado['data_hora_compra'].dt.weekday < 5

# hora_periodo: classificar hora da compra
def classificar_periodo(hora):
    if 0 <= hora < 6:
        return 'madrugada'
    elif 6 <= hora < 12:
        return 'manhã'
    elif 12 <= hora < 18:
        return 'tarde'
    else:
        return 'noite'

df_curado['hora_periodo'] = df_curado['data_hora_compra'].dt.hour.apply(classificar_periodo)

In [61]:
# 8. Verifica a data da primeira compra por cliente
primeiras_compras = df_curado.groupby('fk_contact')['data_hora_compra'].min().reset_index()
primeiras_compras['primeira_compra'] = True

# Faz merge com o dataframe curado
df_curado = df_curado.merge(primeiras_compras, on=['fk_contact', 'data_hora_compra'], how='left')

# Preenche os valores nulos com False (ou seja, não é a primeira compra)
df_curado['primeira_compra'] = df_curado['primeira_compra'].fillna(False)

C:\Users\igor cardoso\AppData\Local\Temp\ipykernel_6940\1823935396.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_curado['primeira_compra'] = df_curado['primeira_compra'].fillna(False)


In [62]:
# 9. Renomear colunas para nomes mais claros
df_curado.rename(columns={
    'nk_ota_localizer_id': 'order_id',
    'fk_contact': 'client_id',
    'place_origin_departure': 'origin_departure',
    'place_destination_departure': 'destination_departure',
    'place_origin_return': 'origin_return',
    'place_destination_return': 'destination_return',
    'fk_departure_ota_bus_company': 'bus_company_departure',
    'fk_return_ota_bus_company': 'bus_company_return',
    'gmv_success': 'total_value',
    'total_tickets_quantity_success': 'tickets_quantity',
    'data_hora_compra': 'purchase_datetime',
    'classificacao_viagem': 'trip_type',
    'tipo_compra': 'purchase_type',
    'sem_retorno_flag': 'no_return_flag',
    'compra_dia_util': 'purchase_weekday_flag',
    'hora_periodo': 'purchase_time_period',
    'primeira_compra': 'first_purchase_flag'
}, inplace=True)

# 10. Reordenar colunas para melhor organização
nova_ordem = [
    # Informações da compra
    'purchase_datetime', 'order_id', 'client_id', 'purchase_weekday_flag', 'purchase_time_period',
    # Informações do cliente
    'first_purchase_flag',
    # Informações da passagem
    'purchase_type', 'tickets_quantity', 'total_value', 'trip_type', 'no_return_flag',
    # Destinos e empresas
    'origin_departure', 'destination_departure', 'origin_return', 'destination_return',
    'bus_company_departure', 'bus_company_return'
]


df_curado = df_curado[nova_ordem]

In [63]:
df_curado.head()

,purchase_datetime,order_id,client_id,purchase_weekday_flag,purchase_time_period,first_purchase_flag,purchase_type,tickets_quantity,total_value,trip_type,no_return_flag,origin_departure,destination_departure,origin_return,destination_return,bus_company_departure,bus_company_return
0,2018-12-26 15:33:35,bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,True,tarde,True,individual,1,89.09,ida,True,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,50e9a8665b62c8d68bccc77c7c92431a1aa26ccbd38ed4...,NaN,NaN,8527a891e224136950ff32ca212b45bc93f69fbb801c3b...,NaN
1,2018-12-05 15:07:57,5432f12612dd5d749b3be880e779989cf63b5efa4bcc4e...,37228485e0dc83d84d1bcd1bef3dc632301bf6cb22c8b5...,True,tarde,False,individual,1,155.97,ida,True,10e4e7caf8b078429bb1c80b1a10118ac6f963eff098fd...,e6d41d208672a4e50b86d959f4a6254975e6fb9b088116...,NaN,NaN,36ebe205bcdfc499a25e6923f4450fa8d48196ceb4fa0c...,NaN
2,2018-12-21 18:41:54,fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,True,noite,False,individual,1,121.99,ida,True,7688b6ef52555962d008fff894223582c484517cea7da4...,8c1f1046219ddd216a023f792356ddf127fce372a72ec9...,NaN,NaN,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,NaN
3,2018-12-06 14:01:38,4dc44a6dd592b702feccb493d192210c86965aee684529...,ab3251a2be0f69713b8f97b0e9d1579e31551f4fd4facf...,True,tarde,False,individual,1,55.22,ida,True,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,d6acb3c1a79e57bcc03d976cb4d98f56edccd4cf426392...,NaN,NaN,5f9c4ab08cac7457e9111a30e4664920607ea2c115a143...,NaN
4,2021-02-23 20:08:25,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,ceea0de820a6379f2c4215bddaec66c33994b304607e56...,True,noite,True,individual,1,45.31,ida,True,7688b6ef52555962d008fff894223582c484517cea7da4...,23765fc69c4e3c0b10f5d15471dc2245e2a19af16b513f...,NaN,NaN,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,NaN


In [65]:
# Agrupar por período e listar as horas únicas que aparecem em cada período
horas_por_periodo = df_curado.groupby('purchase_time_period')['purchase_datetime'].apply(lambda x: sorted(x.dt.hour.unique()))

print("Horas por período:")
for periodo, horas in horas_por_periodo.items():
    print(f"{periodo}: {horas}")


Horas por período:
madrugada: [np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5)]
manhã: [np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11)]
noite: [np.int32(18), np.int32(19), np.int32(20), np.int32(21), np.int32(22), np.int32(23)]
tarde: [np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17)]
